In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds
import numpy as np

dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

In [5]:
def preprocess_dataset(dataset):
    texts, labels = [], []
    for text, label in dataset:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    return texts, np.array(labels)

X_train, y_train = preprocess_dataset(train_data)
X_test, y_test = preprocess_dataset(test_data)

2025-05-29 23:28:48.798662: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-05-29 23:28:49.870030: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-29 23:28:50.966163: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 200
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [7]:
model = Sequential([
    Embedding(10000, 128, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_data=(X_test_pad, y_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 88s 111ms/step - accuracy: 0.5285 - loss: 0.6887 - val_accuracy: 0.6424 - val_loss: 0.6391


In [9]:
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.6449 - loss: 0.6375
Test Accuracy: 64.24%


In [10]:
def predict_opinion(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    prediction = model.predict(padded_sequence)
    return "Positive" if prediction >= 0.5 else "Negative"

print(predict_opinion("This movie was absolutely fantastic! The acting was top notch and the story was compelling."))
print(predict_opinion("I hated every second of this film. The plot was nonexistent and the acting was awful."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Positive
